In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import fastparquet
import pickle
pd.set_option('display.max_columns', None)

In [2]:
ratings = pd.read_parquet("ratings.parquet", engine="fastparquet")

In [3]:
ratings['user_id'].nunique(), ratings['anime_id'].nunique()

(989042, 12984)

In [4]:
ratings.groupby("anime_id")['anime_id'].value_counts().describe()

count     12984.000000
mean       9951.983056
std       31842.353811
min           1.000000
25%         119.000000
50%         815.500000
75%        5120.500000
max      555102.000000
Name: count, dtype: float64

In [5]:
ratings.groupby("user_id")['user_id'].value_counts().describe()

count    989042.000000
mean        130.647994
std         185.555357
min           1.000000
25%          20.000000
50%          74.000000
75%         172.000000
max       11621.000000
Name: count, dtype: float64

In [23]:
a_lb = ratings.groupby("anime_id")['anime_id'].value_counts().quantile(.85)
a_ub = ratings.groupby("anime_id")['anime_id'].value_counts().quantile(.90)
print(a_lb, a_ub)

12904.199999999997 23167.20000000001


In [36]:
u_lb = ratings.groupby("user_id")['user_id'].value_counts().quantile(.65)
u_ub = ratings.groupby("user_id")['user_id'].value_counts().quantile(.90)
print(u_lb, u_ub)

124.0 310.0


In [37]:
_ = ratings.groupby("anime_id").filter(lambda x: a_lb <= len(x) <= a_ub)
valid_ratings = _.groupby("user_id").filter(lambda x: u_lb <= len(x) <= u_ub)

In [38]:
valid_ratings

,user_id,anime_id,favorite,score
2138,----shia,1004,0,8.0
2139,----shia,10049,0,6.0
2147,----shia,102,0,6.0
2159,----shia,10460,0,7.0
2168,----shia,10681,0,6.0
...,...,...,...,...
129203843,wieczorny,9331,0,4.0
129203847,wieczorny,9479,0,5.0
129203849,wieczorny,9587,0,3.0
129203854,wieczorny,9750,0,4.0


In [39]:
valid_ratings['user_id'].nunique(), valid_ratings['anime_id'].nunique()

(7473, 649)

In [40]:
valid_train, valid_test = train_test_split(
    valid_ratings,
    train_size=0.9,
    stratify=valid_ratings["user_id"],
    random_state=42,
)

In [41]:
valid_train.shape, valid_test.shape

((1139585, 4), (126621, 4))

In [42]:
valid_train.groupby("anime_id")['anime_id'].value_counts().sort_values(ascending=False)

anime_id
26441    3175
20457    3012
33433    3003
14527    2971
21353    2950
         ... 
40615     490
42940     487
43697     485
44200     443
37585     282
Name: count, Length: 649, dtype: int64

In [43]:
valid_train.groupby("user_id")['user_id'].value_counts().sort_values(ascending=False)

user_id
jlh_mirtis          279
darksuzaku          279
gridironotaku       279
neverarine          279
norbi22             279
                   ... 
mario_bros81        112
ibrahim3804539el    112
the_riadorana       112
the_nameless_one    112
skurd               112
Name: count, Length: 7473, dtype: int64

In [44]:
valid_train['user_id'].nunique(), valid_train['anime_id'].nunique()

(7473, 649)

In [45]:
valid_train.to_parquet("valid_train.parquet", engine="fastparquet")
valid_test.to_parquet("valid_test.parquet", engine="fastparquet")

In [46]:
valid_train

,user_id,anime_id,favorite,score
45918825,exiax,6030,0,3.0
54333381,mad-matt,14669,0,8.0
117565410,yuzuaihara700,5112,0,8.0
101626350,shikikun,21743,0,9.0
87559943,percutient,2164,0,7.0
...,...,...,...,...
10612029,arvin_1219,25519,0,8.0
101535071,shigenojr,22507,0,9.0
76206881,mtsrhea,153,0,6.0
18103483,chamillie,325,0,1.0


In [47]:
int_mat = valid_train.pivot_table(index="user_id", columns="anime_id", values="score", dropna=True)

In [48]:
int_mat

anime_id,15,22,25,26,27,50,53,63,65,67,74,76,80,81,90,94,97,98,101,102,114,123,132,133,135,137,138,139,147,149,153,157,160,166,169,182,190,193,196,241,243,248,250,257,265,267,287,325,329,371,372,379,390,401,405,408,413,440,449,450,451,452,460,461,462,463,464,465,477,479,487,522,532,534,558,587,594,601,617,627,658,666,687,710,721,732,761,762,770,779,781,834,855,856,859,885,908,949,969,974,985,986,987,996,1004,1015,1030,1074,1121,1142,1239,1250,1281,1303,1361,1365,1462,1498,1536,1562,1566,1579,1594,1606,1669,1690,1699,1709,1722,1816,1827,1894,1965,2116,2131,2154,2164,2201,2248,2397,2403,2404,2494,2508,2596,2685,2899,2927,2952,3087,3091,3225,3228,3229,3230,3299,3342,3358,3467,3572,3655,3667,3701,3713,3972,4026,4087,4106,4177,4192,4551,4581,4789,4866,4884,4970,4981,5079,5112,5118,5150,5262,5337,5342,5355,5365,5391,5507,5678,5690,5764,5781,5941,5962,6030,6127,6171,6178,6336,6377,6444,6572,6574,6772,6793,6802,6895,6974,6987,7017,7465,7590,7627,7662,7695,7858,8426,8476,8479,8514,8536,8577,8740,8768,8857,8876,8915,8934,9047,9107,9136,9331,9479,9587,9736,9750,9776,9790,9834,9981,10049,10073,10083,10119,10152,10278,10298,10357,10378,10447,10456,10460,10491,10507,10638,10647,10681,10686,10798,10805,10851,11077,11113,11123,11241,11739,11743,11751,11783,11837,12031,12431,12565,12679,12685,12711,12783,12883,12979,13055,13333,13391,13403,13411,13851,13939,14093,14175,14189,14293,14407,14527,14669,14751,14753,14765,14807,14921,15059,15119,15199,15201,15437,15565,15613,15751,15771,15775,15879,15911,16005,16099,16157,16273,16355,16397,16444,16694,16700,16866,16890,16916,17082,17121,17389,17535,17741,17827,17831,17909,18295,18393,18441,18617,18661,18767,18893,19021,19117,19151,19285,19365,19383,19511,19703,19759,20431,20449,20457,20709,20931,20939,20973,21185,21329,21353,21473,21563,21653,21667,21681,21743,21797,22097,22123,22273,22429,22507,22547,22835,22839,23225,23269,23311,23333,23383,23775,23777,24031,24037,24211,24277,24627,24687,24705,24873,24893,25161,25303,25429,25517,25519,25649,25687,26213,26351,26441,26443,27663,27829,27891,27957,28069,28387,28391,28405,28619,28771,28833,28883,28957,29027,29101,29589,29787,29974,29976,30014,30091,30127,30206,30230,30279,30321,30365,30383,30385,30415,30485,30709,30711,30782,30806,30901,30991,31051,31098,31138,31174,31234,31318,31326,31327,31380,31430,31483,31500,31588,31629,31630,31665,31680,31706,31711,31715,31771,31812,31914,32032,32071,32083,32086,32108,32245,32271,32360,32365,32366,32407,32491,32494,32551,32553,32607,32664,32668,32673,32696,32698,32924,32930,32936,32948,32977,32981,32983,33003,33012,33036,33051,33080,33091,33095,33142,33203,33222,33354,33421,33433,33513,33524,33558,33573,33654,33834,33985,34034,34051,34100,34102,34126,34136,34152,34213,34262,34277,34300,34350,34392,34414,34494,34525,34537,34540,34541,34544,34591,34620,34658,34855,34973,35015,35145,35241,35249,35677,35756,35806,35821,35823,35828,35889,35983,36039,36124,36214,36316,36317,36516,36616,36632,36652,36653,36688,36754,36792,36817,36903,36906,36936,36990,37007,37021,37033,37078,37202,37208,37377,37379,37426,37440,37449,37492,37525,37579,37585,37597,37621,37651,37719,37722,37806,37823,37920,37926,37986,38256,38328,38422,38476,38619,38699,38735,38924,38935,38936,38940,38959,39063,39071,39199,39486,39651,39705,39808,40128,40206,40215,40269,40359,40392,40453,40532,40615,40623,40679,40685,40746,40870,40907,40936,40960,40974,41265,41392,41783,42192,42205,42395,42544,42590,42627,42940,43697,44200,44807,44881,44961,45055,46093,46985,48406,48471,48488,48569,48590,48707,49357,49605
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [49]:
user_encode = {usrid: index for index, usrid in enumerate(int_mat.index)}
user_decode = {index: usrid for index, usrid in enumerate(int_mat.index)}

In [50]:
anime_encode = {animeid: index for index, animeid in enumerate(int_mat.columns)}
anime_decode = {index: animeid for index, animeid in enumerate(int_mat.columns)}

In [51]:
int_mat.reset_index(drop=True, inplace=True)
int_mat.columns = [str(i) for i in range(int_mat.shape[1])]
int_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648
0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,6.0,NaN,NaN,NaN,NaN,6.0,NaN,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,8.0,6.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,8.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,4.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,5.0,7.0,8.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,6.0,7.0,6.0,NaN,NaN,NaN,8.0,NaN,NaN,7.0,NaN,NaN,8.0,6.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,7.0,7.0,7.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,7.0,4.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,8.0,6.0,NaN,NaN,5.0,7.0,NaN,6.0,6.0,NaN,NaN,NaN,NaN,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,3.0,5.0,7.0,NaN,7.0,NaN,NaN,6.0,NaN,NaN,7.0,NaN,NaN,6.0,NaN,NaN,NaN,5.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,7.0,6.0,NaN,NaN,NaN,NaN,8.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN

In [52]:
int_mat.to_parquet("interactions.parquet", engine="fastparquet")

In [53]:
with open("encode_decode/user_encode.pkl", "wb") as f:
    pickle.dump(user_encode, f)
with open("encode_decode/user_decode.pkl", "wb") as f:
    pickle.dump(user_decode, f)

with open("encode_decode/anime_encode.pkl", "wb") as f:
    pickle.dump(anime_encode, f)
with open("encode_decode/anime_decode.pkl", "wb") as f:
    pickle.dump(anime_decode, f)